In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 設定目前程式資料夾路徑(務必自行更改)

In [ ]:
folder="/content/drive/MyDrive/解密AI黑盒子分享/"

## 載入所需套件

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
twfont1=plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")
from IPython import display

## 載入pytorch相關套件

In [ ]:
import torch
#建立神經網路用
import torch.nn as nn
import torch.nn.functional as F
#載入優化器
from torch import optim
#預處理資料用
from torch.utils.data import Dataset,DataLoader

## 建立DataSet類別

In [ ]:
class MyDataset(Dataset): 
  def __init__(self,Train=True):
    if Train==True:
      data=np.loadtxt(folder+"資料集/MNIST/mnist_train.csv",delimiter=",")
    else:
      data=np.loadtxt(folder+"資料集/MNIST/mnist_test.csv",delimiter=",")
    self.data = torch.tensor(data[:,1:]/255)
    self.label = torch.tensor(data[:,0])
  def __getitem__(self, index): 
    return self.data[index], self.label[index]
  def __len__(self): 
    return len(self.data)

## 實體化訓練和測試DataSet和DataLoader

In [ ]:
TrainDS=MyDataset(Train=True)
TestDS=MyDataset(Train=False)
TrainDL=DataLoader(dataset=TrainDS,batch_size=200,shuffle=True)
TestDL=DataLoader(dataset=TestDS,batch_size=len(TestDS),shuffle=True)

## 顯示手寫數字照片

## 建立神經網路類別

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):


    return x

## 實體化神經網路，指定損失函數和優化器

In [ ]:
net=Net()
optimizer = optim.SGD(net.parameters(), lr=0.01)
LossFun=nn.CrossEntropyLoss()
Step_L=[]

## 開始訓練神經網路

In [ ]:
net.train()
for i in range(150):
  for data,label in TrainDL:
    optimizer.zero_grad()
    Yh=net(data.float())
    loss=LossFun(Yh,label.long())
    Step_L.append(float(loss))
    loss.backward()
    optimizer.step()
  print("訓練回合：",i+1,"損失值：",loss)
  display.clear_output(wait=True)

## 觀察訓練過程中的損失函數Loss變化

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Loss隨訓練次數的變化",fontproperties=twfont1,fontsize=20) 
plt.xlabel("訓練次數",fontproperties=twfont1,fontsize=20) 
plt.ylabel("Loss值",fontproperties=twfont1,fontsize=20)
plt.plot(Step_L,":o")
plt.show()

In [ ]:
net.eval()
with torch.no_grad():
  data,label=next(TestDL.__iter__())
  output = net(data.float())
print("驗證資料準確度：",(torch.argmax(output,dim=1)==label).sum().item()*100/len(TestDS),"%")